# Code Agents with `smolagents 🤗`

This notebook demonstrates how to use `smolagents` to create and run different types of AI agents.

We'll cover:
- Basic code agents
- Tool-calling agents
- RAG systems
- Web interaction
- SQL query generation

`smolagents` provides a lightweight framework for building agents that can execute code and use tools.

## Setup Environment

First we configure the environment:
- Load environment variables (API keys, etc)
- Enable automatic reloading for development
- Import required libraries
- Reset the notebook state

In [ ]:
# Default Setup Cell.
# It imports environment variables, define 'devtools.debug" as a buildin, set PYTHONPATH and autorelaod
# Copy it in other Notebooks


from dotenv import load_dotenv
from rich import print

load_dotenv(verbose=True)

%load_ext autoreload
%autoreload 2
%reset -f

In [ ]:
from typing import Optional

from genai_blueprint.utils.config_mngr import global_config_reload
from genai_tk.core.llm_factory import LlmFactory
from IPython.display import Markdown, display
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    VisitWebpageTool,
    WebSearchTool,
    tool,
)

global_config_reload()

MODEL_ID = None
# llm_factory = LlmFactory(llm_id=MODEL_ID, llm_params={"temperature": 0.7})
llm_factory = LlmFactory(llm_tag="azure", llm_params={"temperature": 0.7})
llm = llm_factory.get_smolagent_model()
print(f"Using model: {llm_factory.get_id()}")

## Basic Code Agent

Let's create a simple code agent that can:
- Answer questions
- Use web search when needed
- Execute Python code

We'll ask it to calculate how long it would take a leopard to run across a famous Paris bridge.

In [ ]:
agent = CodeAgent(tools=[WebSearchTool()], model=llm)
agent.run("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")

## Tool-Calling Agent

Now we'll create an agent that can call custom tools.

We define a simple weather tool that:
- Takes a location as input
- Returns a fixed weather report

The agent will decide when to use this tool based on the query.

In [ ]:
@tool
def get_weather(location: str, celsius: Optional[bool] = False) -> str:
    """Get weather in the next days at given location.
    Secretly this tool does not care about the location, it hates the weather everywhere.

    Args:
        location: the location
        celsius: the temperature
    """
    return "The weather is UNGODLY with torrential rains and temperatures below -10°C"


agent = ToolCallingAgent(tools=[get_weather], model=llm)
print(agent.run("What's the weather like in Paris?"))

## RAG System

Here we combine `smolagents` with LangChain to create a Retrieval Augmented Generation (RAG) system that:

1. Loads and chunks a text document
2. Stores it in a vector store
3. Creates a retriever tool
4. Uses an agent to answer questions based on the document

We'll use the US State of the Union address as our test document.

In [ ]:
from genai_tk.core.embeddings_factory import EmbeddingsFactory
from genai_tk.core.embeddings_store import EmbeddingsStore
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from genai_blueprint.wip.smolagents_chain import RetrieverTool

# Usual RAG stuff : load document, split it, and add chunks into a vectorstore

documents = TextLoader("use_case_data/other/state_of_the_union.txt").load()
texts = RecursiveCharacterTextSplitter(chunk_size=2000).split_documents(documents)
vs = EmbeddingsStore(id="InMemory", embeddings_factory=EmbeddingsFactory()).vector_store
_ = vs.add_documents(texts)
vs_retriever = vs.as_retriever(k=10)


# Initialize agent
def create_rag_agent(retriever):
    retriever_tool = RetrieverTool(retriever)
    return CodeAgent(tools=[retriever_tool], model=llm, max_steps=4)


agent = create_rag_agent(vs_retriever)
response = agent.run("What did the president say about Ketanji Brown Jackson")
print(response)

## Understanding the Agent

Let's inspect the agent's prompt templates to understand how it works.

The templates define:
- How the agent thinks
- How it formats responses
- Its decision-making process

In [ ]:
agent.prompt_templates

In [ ]:
display(Markdown(agent.prompt_templates["final_answer"]))

## Web Interaction

Now we'll create an agent that can:
- Visit web pages
- Extract information
- Answer questions based on web content

We'll ask it to find France's current prime minister.

In [ ]:
# search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])

# agent = CodeAgent(tools=[search_tool], model=HfApiModel())

# agent.run("How many more blocks (also denoted as layers) are in BERT base encoder compared to the encoder from the architecture proposed in Attention is All You Need?")

In [ ]:
agent = CodeAgent(
    tools=[VisitWebpageTool()],
    model=llm,
    additional_authorized_imports=["requests", "markdownify", "bs4"],
    # use_e2b_executor=True,
)

agent.run("Qui est l'actuel premier ministre en France ?")

## SQL Query Generation

Finally, we'll create an agent that can:
- Understand natural language questions
- Generate SQL queries
- Execute them against a database
- Correct errors automatically

We'll use a simple in-memory SQLite database with restaurant receipt data.

In [ ]:
from sqlalchemy import (
    Column,
    Float,
    Integer,
    MetaData,
    String,
    Table,
    create_engine,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
    {"receipt_id": 5, "customer_name": "John Doe", "price": 100.00, "tip": 10.00},
]
for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

table_name = "waiters"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("waiter_name", String(16), primary_key=True),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Watts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
]
for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [ ]:
updated_description = """Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:"""

inspector = inspect(engine)
for table in ["receipts", "waiters"]:
    columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table)]

    table_description = f"Table '{table}':\n"

    table_description += "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
    updated_description += "\n\n" + table_description

print(updated_description)

In [ ]:
from smolagents import tool


@tool
def sql_engine(query: str) -> str:
    """Allows you to perform SQL queries.

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output


sql_engine.description = updated_description

In [ ]:
agent = CodeAgent(tools=[sql_engine], model=llm)
agent.run("Can you give me the name of the client who got the most expensive receipt?")